## 贝叶斯公式

朴素贝叶斯算法是一个典型的统计学习方法，主要理论基础就是一个贝叶斯公式，贝叶斯公式的基本定义如下：

$$P(Y_k|X)=\frac {P(XY_k)} {P(X)}=\frac {P(Y_k)P(X|Y_k)} {\underset {j}{\mathop{\sum}}P(Y_j)P(X|Y_j)}$$

这个公式看上去简单，但它却能总结历史，预知未来。公式的右边是总结历史，公式的左边是预知未来，如果把$Y$看成类别，$X$看成特征，$P(Y_k|X)$就是在已知特征$X$的情况下求$Y_k$类别的概率，而对$P(Y_k|X)$的计算又全部转化到类别$Y_k$的特征分布上来

## 贝叶斯算法


1. 设$x=\{a_1,a_2,...a_m\}$为一个待分类项，而每个$a$为$x$的一个特征属性。
2. 有类别集合$C=\{y_1,y_2,y_n\}$
3. 计算$P(y_1|x),P(y_2|x),...P(y_n|x)$
4. 如果$P(y_k|x)=max\{P(y_1|x),P(y_2|x),...P(y_n|x)\}$，则$x\in y_k$

那么现在的关键就是如何计算第3步中的各个条件概率。朴素贝叶斯算法做了一假设：“朴素的认为各个特征相互独立”：

1. 找到一个已知分类的待分类项集合，这个集合叫做训练样本集。
2. 统计得到在各类别下各个特征属性的条件概率估计。即$P(a_1|y_1),P(a_2|y_1)...P(a_m|y_1);...P(a_1|y_n),P(a_2|y_n)...P(a_m|y_n)$
3. 如果各个特征属性是条件独立的，则根据贝叶斯定理有如下推导：

$$P(y_i|x)=\frac {P(x|y_i)P(y_i)} {P(x)}$$

因为分母对于所有类别为常数，因为我们只要将分子最大化皆可。又因为各特征属性是条件独立的，所以有：

$${P(x|y_i)P(y_i)}= P(a_1|y_i)P(a_2|y_i)...P(a_m|y_i)P(y_i)=P(y_i) \prod_{j=1}^m P(a_j|y_i)$$


贝叶斯公式推导能够成立有个重要前提，就是各个证据（evidence）不能为0。也即对于任意特征$a_i$，$P(a_i)$不能为0。而现实情况中某些特征未出现在测试集中的情况是可以发生的。因此实现上通常要做一些小的处理，例如把所有计数进行+1（加法平滑(additive smoothing，又叫拉普拉斯平滑(Laplace smothing)）。而如果通过增加一个大于0的可调参数alpha进行平滑，就叫Lidstone平滑

最后，我们知道，当特征很多的时候，大量小数值的小数乘法会有溢出风险。因此，通常的实现都是将其转换为log：

$log[P(Y)*P(a_1|Y)*P(a_2|Y)...P(a_m|Y)] = log[P(Y)]+log[P(a_i|Y)] + ... +log[P(a_m|Y)]$

将乘法转换为加法，就彻底避免了乘法溢出风险

## 高斯模型

有些特征可能是连续型变量，比如说人的身高，物体的长度，这些特征可以转换成离散型的值，比如如果身高在160cm以下，特征值为1；在160cm和170cm之间，特征值为2；在170cm之上，特征值为3。也可以这样转换，将身高转换为3个特征，分别是f1、f2、f3，如果身高是160cm以下，这三个特征的值分别是1、0、0，若身高在170cm之上，这三个特征的值分别是0、0、1。不过这些方式都不够细腻，高斯模型可以解决这个问题。高斯模型假设特征的条件分布符合高斯分布，也就是：

$$P(x_{i}|y_{k}) = \frac{1}{\sqrt{2\pi\sigma_{y_{k}}^{2}}}exp( -\frac{(x_{i}-\mu_{y_{k}})^2}  {2\sigma_{y_{k}}^{2}})$$

In [4]:
from sklearn import datasets
iris = datasets.load_iris()
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(iris.data, iris.target)
clf.predict(iris.data[[0]])

array([0])

## 多项式模型

该模型常用于文本分类，特征是单词，值是单词的出现次数

$$P(x_{i}|y_{k}) = \frac{N_{y_{k}x_{i}}+\alpha}{N_{y_{k}}+\alpha n}$$

其中，$N_{y_{k}x_{i}}$是类别$y_k$下特征$x_i$出现的总次数；$N_{y_{k}}$是类别$y_k$下所有特征出现的总次数。对应到文本分类里，如果单词word在一篇分类为label1的文档中出现了5次，那么$N_{label1,word}$的值会增加5。如果是去除了重复单词的，那么$N_{label1,word}$的值会增加1。n是特征的数量，在文本分类中就是去重后的所有单词的数量。$\alpha$的取值范围是[0,1]，比较常见的是取值为1。

待预测样本中的特征$x_i$训练时可能没有出现，如果没有出现，则$N_{y_{k}x_{i}}$值为0，如果直接拿来计算该样本属于某个分类的概率，结果都将是0。在分子中加入$\alpha$，在分母中加入$\alpha n$可以解决这个问题

In [5]:
import numpy as np
X = np.random.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)
print(clf.predict(X[[2]]))


[3]


值得注意的是，多项式模型在训练一个数据集结束后可以继续训练其他数据集而无需将两个数据集放在一起进行训练。在sklearn中，MultinomialNB()类的partial_fit()方法可以进行这种训练。这种方式特别适合于训练集大到内存无法一次性放入的情况。

## 伯努利模型


在伯努利模型中，每个特征的取值是布尔型的，即true和false，或者1和0。在文本分类中，就是一个特征有没有在一个文档中出现。

如果特征值$x_i$值为1,那么

$$P(x_{i}|y_{k}) = P(x_{i}=1|y_{k})$$

如果特征值$x_i$值为0,那么

$$P(x_{i}|y_{k}) = 1-P(x_{i}=1|y_{k})$$

这意味着，“没有某个特征”也是一个特征。

In [6]:
import numpy as np
X = np.random.randint(2, size=(6, 100))
Y = np.array([1, 2, 3, 4, 4, 5])
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X, Y)
print(clf.predict(X[[2]]))

[3]
